# Equivalence of several methods of state evolution by the same unitary #
1. state vector evolution
2. density matrix evolution
3. superoperator evolution

In [3]:
import numpy as np
from qiskit.quantum_info import random_statevector, random_unitary

In [52]:
n_qubits = 1
dim = 2**n_qubits

### Generate a random unitary and state vector ###

In [53]:
U = random_unitary(dim).data
psi = random_statevector(dim).data.reshape(-1, 1)
psi_dm = psi @ psi.conj().T

## 1. state vector evolution ##
$$ U |\psi> = |\phi> $$
And the density matrix for $|\phi>$ is: 
$$|\phi><\phi|$$

In [62]:
phi = U @ psi
phi_dm1 = phi @ phi.conj().T

## 2. density matrix evolution
$$ U |\phi><\phi| U^\dagger = |\phi><\phi| $$

In [63]:
phi_dm2 = U @ psi_dm @ U.conj().T

## 3. superoperator evolution ##

Using the "[vec trick](https://en.wikipedia.org/wiki/Kronecker_product#Matrix_equations)" we can start from density matrix evolution and get:
$$
U |\psi><\psi| U^\dagger = (U^* \otimes U) \text{vec}(|\psi><\psi|)
$$

Where $(U^* \otimes U)$ represents a superoperator and we define $\text{vec}()$ to be the function that vectorizes a density matrix in column-order. We can also define $\text{vec}^{-1}()$ to be the function that undoes the $\text{vec}()$ operation and converts back to a density matrix.


In [64]:
vec = lambda X : X.reshape(-1, 1, order='F')
vec_inverse = lambda X : X.reshape(dim, dim, order='F')

In [65]:
superop = np.kron(U.conj(), U)
phi_dm3 = vec_inverse(superop @ vec(psi_dm))

Note that if instead we define $\text{vec}_{row}()$ such that it uses row-order then it changes to the following:
$$
U |\psi><\psi| U^\dagger = (U \otimes U^*) \text{vec}_{row}(|\psi><\psi|)
$$

In [74]:
vec_row = lambda X : X.reshape(-1, 1)
vec_row_inverse = lambda X : X.reshape(dim, dim)

In [75]:
superop = np.kron(U, U.conj())
phi_dm4 = vec_row_inverse(superop @ vec_row(psi_dm))

## Demonstration of equivalence ##

In [76]:
print(np.allclose(phi_dm1, phi_dm2, phi_dm3, phi_dm4))

True


In [80]:
print("psi:\n", psi, "\n")
print("U:\n", U, "\n")
print("phi_dm:\n", phi_dm1)

psi:
 [[ 0.21189616+0.84819737j]
 [-0.38023996+0.30179267j]] 

U:
 [[-0.18007542-0.23495121j  0.31843593-0.90053836j]
 [ 0.27580523-0.91449562j -0.28415967+0.08296089j]] 

phi_dm:
 [[0.1529275 -3.24267695e-18j 0.26777424+2.40494561e-01j]
 [0.26777424-2.40494561e-01j 0.8470725 -2.91400810e-18j]]
